# Practical 9: Circularity Intervetions

Objectives
- Learn how to create scenarios in IO 
- Assess the impacts of circularity interventions
- Conduct a novel analysis 

Download the the MR_HIOT data from: https://zenodo.org/record/7244919 (n.b. ignore files that start with MR_HSUT or MR_HUSE )

In [25]:
# Import packages
import pandas as pd
import numpy as np

### 1 Import the data

Import the following data

- MR_HIOT_2011_v3_3_18_xx_principal_production.csv indicates the principal production of the productive activities;
- MR_HIOT_2011_v3_3_18_xx_by_product_technology.csv indicates the matrix of the uses where by-products are included with a negative sign;
- MR_HIOT_2011_v3_3_18_xx_FD.csv includes the consumption of final consumers
- MR_HIOT_2011_v3_3_18_xx_stock_to_waste.csv includes the demand of waste treatment services to treat the stock reduction
- MR_HIOT_2011_v3_3_18_xx_extensions.csv includes the extensions as reported in the HSUTs. Very minor changes are applied


In this practical you will only use a few of the imported datasets, this exercise is to help you understand the variety of the data. If you so wish, you may also perform additional analysis with the imported datasets 

1.1 Inter-industry matrix

In [26]:
Z = pd.read_csv("data/MR_HIOT_2011_v3_3_18_by_product_technology.csv", index_col=[0,1,2,3,4], header=[0,1,2,3])

1.2 Final demand

In [27]:
Y  = pd.read_csv("data/MR_HIOT_2011_v3_3_18_FD.csv", index_col=[0,1,2,3,4], header=[0,1,2,3])

1.3 Extensions

In [28]:
extensions = pd.ExcelFile("data/MR_HIOT_2011_v3_3_18_extensions.xlsx")

In [29]:
extensions.sheet_names

['intro',
 'resource_act',
 'resource_FD',
 'Land_act',
 'Land_FD',
 'Emiss_act',
 'Emiss_FD',
 'Emis_unreg_w_act',
 'Emis_unreg_w_FD',
 'waste_sup_act',
 'waste_sup_FD',
 'waste_use_act',
 'waste_use_FD',
 'pack_sup_waste_act',
 'pack_sup_waste_fd',
 'pack_use_waste_act',
 'pack_use_waste_fd',
 'mach_sup_waste_act',
 'mach_sup_waste_fd',
 'mach_use_waste_act',
 'mach_use_waste_fd',
 'waste_from_stocks',
 'stock_addition_act',
 'stock_addition_fd',
 'crop_res_act',
 'crop_res_FD',
 'VA_act']

1.4 Resource extraction

In [30]:
# Resource extraction matrix of Z
RE = extensions.parse(sheet_name="resource_act", index_col=[0,1], header=[0,1,2,3]) 
# Resource extraction matrix of Y
RE_FD = extensions.parse(sheet_name="resource_FD", index_col=[0,1], header=[0,1,2,3]) 

1.5 Waste supply and use accounts

In [31]:
# Waste supply matrix of Z
WS = extensions.parse(sheet_name="waste_sup_act", index_col=[0,1], header=[0,1,2,3]) 
# Waste supply matrix of Y
WS_FD = extensions.parse(sheet_name="waste_sup_FD", index_col=[0,1], header=[0,1,2,3]) 

# Waste use matrix of Z
WU = extensions.parse(sheet_name="waste_use_act", index_col=[0,1], header=[0,1,2,3]) 
# Waste use matrix of Y => This is all 0's so it can also be ignored
WU_FD = extensions.parse(sheet_name="waste_use_FD", index_col=[0,1], header=[0,1,2,3]) 

1.6 Stock additions

In [32]:
# Stock addition matrix of Z
SA = extensions.parse(sheet_name="stock_addition_act", index_col=[0,1], header=[0,1,2,3]) 
# Stock addition matrix of Y
SA_FD = extensions.parse(sheet_name="stock_addition_fd", index_col=[0,1], header=[0,1,2,3]) 

1.7 Stock depletion matrix

In [33]:
# Stock depletion matrix
SD = extensions.parse(sheet_name="waste_from_stocks", index_col=[0,1], header=[0,1,2,3])

1.8 Emissions

In [34]:
EM = extensions.parse(sheet_name="Emiss_act", index_col=[0,1,2], header=[0,1,2,3])
EM_FD = extensions.parse(sheet_name="Emiss_FD", index_col=[0,1,2], header=[0,1,2,3])

## Exercise 2: Calculate the MR EEIO variables

### 2.1  Product total output and its diagonal inverse

In [35]:
x = Z.sum(axis=1) + Y.sum(axis=1)

# we make a copy of our product output vector
x_ = x.copy() 

# we divide 1 by the values that are non-0
x_[x_!=0] = 1/x_[x_!=0]

# We diagolize the resulting vector
inv_diag_x = np.diag(x_)

### 2.2 Technical coefficient matrix and the Leontief inverse

In [36]:
A = Z @ inv_diag_x

I = np.eye(A.shape[0])

L = np.linalg.inv(I-A)

In [37]:
pd.concat([pd.Series(L@Y.sum(axis=1), index=x.index), x],axis=1, ignore_index=True).groupby(level=1).sum()

,0,1
Refined Petroleum,4.819022e+09,4.820498e+09
Air transport services (62),5.814680e+05,5.815086e+05
Aluminium and aluminium products,4.478585e+07,4.478860e+07
Aluminium ores and concentrates,1.075348e+08,1.075377e+08
Animal products nec,1.207135e+06,1.207144e+06
...,...,...
"Wood material for treatment, Re-processing of secondary wood material into new wood material",0.000000e+00,0.000000e+00
Wood waste for treatment: incineration,2.186499e+06,2.655406e+06
Wood waste for treatment: landfill,8.501762e+06,9.712888e+06
"Wool, silk-worm cocoons",3.163636e+05,3.162147e+05


### 2.3 Calculate extension intensity

#### 2.3.1 First we sum the following items along their rows
- $W_{sup}$ is the global waste supply
- $W_{rec}$ is the global waste use/recovery 
- $S_{dep}$ is the global stock depletation 

In [38]:
WS_ind_sum = WS.sum(axis=0)
WS_FD_cat_sum = WS_FD.sum(axis=0)

WU_ind_sum = WU.sum(axis=0)
WU_FD_cat_sum = WU_FD.sum(axis=0)

SD_col_sum = SD.sum(axis=0)

### 2.3.2 Calculate the intensity of the summed variables

In [39]:
WS_f = WS_ind_sum @ inv_diag_x
WU_f = WU_ind_sum @ inv_diag_x

### 2.3.3 Calculate the intensity of total resource extraction
N.b. disregard the last 4 rows i.e., oxigen and water items

In [40]:
RE_f = RE.iloc[:-4].sum(axis=0) @ inv_diag_x

## Exercise 3: Create a scenario (i.e., counterfactual IO system) based on your chosen circularity intervention 

In [41]:
Y_ct = Y.copy()

# 1 vector of zeros of the same length of the rows
i_i = np.zeros(Y_ct.shape[0])

In [42]:
Y_ct = Y.copy()

# Mock change 

# Primary change
# Technical change coefficient (e.g., )
kt_pc = 0.5
# Market penetration
kp_pc = 0.3 
# implement primary change
Y_ct.loc[pd.IndexSlice[:,"Motor vehicles, trailers and semi-trailers (34)"],:] *= kt_pc * kp_pc


# Secondary change
# Technical change coefficient
kt_pc = 1.2
# Market penetration
kp_pc = 1 
# implement secondary change
Y_ct.loc[pd.IndexSlice[:,:,:,"C_TDMO"],:] *= kt_pc * kp_pc



In [43]:
# Calculate the counterfactual total product output
x_ct = L @ Y_ct.sum(1)

## Exercise 4: Calculate the global circularity gap and resource extraction of the counterfactual and compare it to the baseline

Circularity gap in absolute terms

$ CG = W_{sup} + S_{dep} - W_{rec}$

Where:
- $CG$ is the circularity gap
- $W_{sup}$ is the global waste supply
- $S_{dep}$ is the global stock depletation 
- $W_{rec}$ is the global waste use/recovery 


Circularity gap index

$ CGI = \frac{CG}{W_{sup} + S_{dep}} \times 100$



Methods from:
 
Aguilar-Hernandez, G. A., Sigüenza-Sanchez, C. P., Donati, F., Merciai, S., Schmidt, J., Rodrigues, J. F., & Tukker, A. (2019). The circularity gap of nations: A multiregional analysis of waste generation, recovery, and stock depletion in 2011. Resources, Conservation and Recycling, 151, 104452.

### 2.1: Calculate the global total waste generation

In [44]:
# Baseline WS
WS_tot = WS.sum().sum() + WS_FD.sum().sum()

# Counterfactual
# intensity vector of WS
f_WS = WS.sum() @ inv_diag_x
# counterfactual WS
WS_ct_tot = f_WS @ x_ct + WS_FD.sum().sum()

WS_tot, WS_ct_tot

(14577617331.961227, 13989057751.61644)

### 2.2: Calculate global total waste use (i.e., recovery)

In [45]:
# Baseline WU
WU_tot = WU.sum().sum() + WU_FD.sum().sum()

# Counterfactual
# intensity vector of WU
f_WU = WU.sum() @ inv_diag_x
# counterfactual WU
WU_ct_tot = f_WU @ x_ct + WU_FD.sum().sum()

WU_tot, WU_ct_tot

(16712487923.123253, 16163003262.56723)

### 2.3: Calculate global total stock degradation

In [46]:
SD_tot = SD.sum().sum()
SD_tot

4736719890.900244

### 2.4: Circularity gap calculations


In [47]:
# CIRCULARITY GAP CALCULATION
circularity_gap = WS_tot + SD_tot - WU_tot
circularity_gap_ct = WS_ct_tot + SD_tot - WU_ct_tot

print(f"Global Circularity Gap: {round(circularity_gap/(WS_tot + SD_tot)*100, 1)} %")
print(f"Global Circularity Gap counterfactual: {round(circularity_gap_ct/(WS_ct_tot + SD_tot)*100, 1)} %")

Global Circularity Gap: 13.5 %
Global Circularity Gap counterfactual: 13.7 %


## Exercise 5: Let's compare now the total resource use

In [48]:
# Baseline
RE_base = RE.iloc[:-3].sum().sum() + RE_FD.iloc[:-3].sum().sum()

# Counterfactual
# intensity vector of RE
f_RE = RE.iloc[:-3].sum() @ inv_diag_x
# counterfactual WU
RE_ct_tot = f_RE @ x_ct + RE_FD.iloc[:-3].sum().sum()

f"Change in resource use: {round((RE_ct_tot-RE_base)/RE_base*100,1)} %"


'Change in resource use: -0.9 %'

## Exercise 6: Reflect on your scenario, what does an improvement in circularity index tells us about sustainability? 